In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd

In [3]:
train_df=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_df=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
train_df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [6]:
test_df.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [7]:
train_df = train_df.drop(['id','location','keyword'], axis=1)
test_df = test_df.drop(['id','location','keyword'], axis=1)

In [8]:
train_df["text"] = train_df["text"].apply(lambda x: x.lower())
test_df["text"] = test_df["text"].apply(lambda x: x.lower())

In [9]:
import re
def remove_URL(text):
    return re.sub(r"https?://\S+|www\.\S+", "", text)

train_df["text"] = train_df["text"].apply(lambda x: remove_URL(x))
test_df["text"] = test_df["text"].apply(lambda x: remove_URL(x))

In [10]:
def remove_html(text):
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)
train_df["text"] = train_df["text"].apply(lambda x: remove_html(x))
test_df["text"] = test_df["text"].apply(lambda x: remove_html(x))

In [11]:
import string
def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))

train_df["text"] = train_df["text"].apply(lambda x:remove_punct(x))
test_df["text"] = test_df["text"].apply(lambda x: remove_punct(x))

In [12]:
import nltk
from nltk import TweetTokenizer

tokenizer = TweetTokenizer()

train_df['tokens'] = [tokenizer.tokenize(item) for item in train_df.text]
test_df['tokens'] = [tokenizer.tokenize(item) for item in test_df.text]

In [13]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_item(item):
    new_item = []
    for x in item:
        x = lemmatizer.lemmatize(x)
        new_item.append(x)
    return " ".join(new_item)

train_df['tokens'] = [lemmatize_item(item) for item in train_df.tokens]
test_df['tokens'] = [lemmatize_item(item) for item in test_df.tokens]

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()


X = vectorizer.fit_transform(train_df.text).toarray()
y = train_df['target']

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [16]:
from sklearn.naive_bayes import MultinomialNB
mb = MultinomialNB()
mb.fit(X_train,y_train)

MultinomialNB()

In [17]:
y_pred = mb.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
acc = accuracy_score(y_test,y_pred)
print("The accuracy of the model is :",acc)

The accuracy of the model is : 0.8025394045534151


In [19]:
cf = confusion_matrix(y_test,y_pred)
cf

array([[1157,  181],
       [ 270,  676]])

In [20]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.86      0.84      1338
           1       0.79      0.71      0.75       946

    accuracy                           0.80      2284
   macro avg       0.80      0.79      0.79      2284
weighted avg       0.80      0.80      0.80      2284



In [21]:
test = vectorizer.transform(test_df.text).toarray()

In [22]:
pred_1 = mb.predict(test)

In [23]:
ss = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
ss.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [24]:
ss['target'] = pred_1
ss.to_csv('submission.csv',index=False)

In [25]:
r =pd.read_csv("submission.csv")
r

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1
